In [ ]:
import pandas as pd
import sqlite3
from urllib.parse import urlparse, parse_qs
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import datetime
from tqdm import tqdm

In [ ]:
con = sqlite3.connect("dataset.sqlite3")
df = pd.read_sql_query("""SELECT publications.is_processed, publications.scholar_id,
                        publications.publication_url,publications.publication_date,
                        publications.citations,
                        publications.author_names, authors.name FROM publications
                        JOIN authors ON publications.scholar_id = authors.scholar_id
                        ;""", con)
# Verify that result of SQL query is stored in the dataframe
print(df.head())
con.close()

In [ ]:
def get_publication_computed_fields(row: pd.Series):
    total_authors = 0
    author_position = 0
    is_present = 0
    publication_year = None
    if row['publication_date'] is not None:
        publication_year = extract_year(row['publication_date'])
    if row['author_names'] is not None:
        name_list =r['author_names'].split(', ')
        total_authors = len(name_list)
        # print(name_list,id)
        closest_match, confidence = process.extractOne(r['name'], name_list,scorer=fuzz.token_set_ratio)
        if confidence>75:
            is_present = 1
            # print(closest_match)
            author_position = name_list.index(closest_match)+1
        else:
            total_authors+=1
            author_position = total_authors
    return {
        'scholar_id': row['scholar_id'],
        'publication_url': row["publication_url"],
        'total_authors': total_authors,
        'author_position': author_position,
        'publication_year': publication_year,
        'is_present': is_present
        }




def extract_year(value):
    try:
        dt = datetime.strptime(value, '%Y/%m/%d')
        return dt.year
    except ValueError:
        try:
            dt = datetime.strptime(value, '%Y')
            return dt.year
        except ValueError:
            return None

In [ ]:
conn = sqlite3.connect('CitationDataset.sqlite3')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

with tqdm(total=len(df)) as pbar:
    for i,r in df.iterrows():
        computed_fields = get_publication_computed_fields(r)
        pbar.update(1)
        # print(computed_fields,r)
        cursor.execute("""
    INSERT INTO
        publications_computed_fields
        (publication_url, author_position, total_authors, author_present, publication_year,scholar_id)
    VALUES
        (:publication_url, :author_position, :total_authors, :is_present, :publication_year,:scholar_id)""", computed_fields)
    
    # cursor.execute("UPDATE publications SET author_position = ?, author_present = ? WHERE id = ?", (author_position,not_found,id))
    # print(author_position,not_found)
# Commit the changes
conn.commit()

conn.close()
print('Successfully executed')

In [ ]:
con = sqlite3.connect("CitationDataset.sqlite3")
publication_computed_df = pd.read_sql_query("""SELECT publications.publication_url, publications.scholar_id,
                        publications.citations,publications_computed_fields.total_authors,
                        publications_computed_fields.author_position, publications_computed_fields.author_present FROM publications
                        JOIN publications_computed_fields ON publications.publication_url = publications_computed_fields.publication_url
                        ;""", con)
scholar_ids = pd.read_sql_query("""select distinct(scholar_id) from publications;""",con)
# Verify that result of SQL query is stored in the dataframe
print(publication_computed_df.head())
con.close()

In [ ]:
def calculate_h_index(works_df:pd.DataFrame):
    works_df.sort_values('citations', inplace=True,ascending=False)
    h_index=0.0
    for i,r in works_df.iterrows():
        if h_index+1 > r['citations']:
            break
        h_index+=1
    return h_index

In [ ]:
#add average position, average authors and calculated h-index
con = sqlite3.connect("/home/somir/Desktop/CitationDataset/CitationDataset.sqlite3")
c = con.cursor()
with tqdm(total=len(scholar_ids)) as pbar:
    for i,r in scholar_ids.iterrows():
        # print(r)
        temp_df = publication_computed_df[publication_computed_df['scholar_id']==r['scholar_id']].copy()
        h_index = calculate_h_index(temp_df)
        avg_authors = temp_df.groupby('scholar_id')['total_authors'].mean()
        avg_author_position = temp_df.groupby('scholar_id')['author_position'].mean()
        # print(h_index,avg_authors.values[0],avg_author_position.values[0])
        con.execute("""
    INSERT INTO
        authors_computed_fields
        (scholar_id, calculated_h_index, avg_authors, avg_position)
    VALUES
        (?, ?, ?, ?)""", (r['scholar_id'],h_index,avg_authors.values[0],avg_author_position.values[0]))
        pbar.update(1)
    con.commit()
    con.close()